In [ ]:
%pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 79.0 MB/s eta 0:00:00


In [ ]:
import re
import numpy as np
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer
import faiss
from openai import OpenAI
import time

In [ ]:
class ArabicRAGGPTold:
    def __init__(self, document_path, api_key):
        self.document_path = document_path
        self.api_key = api_key
        self.initialized = False

        # سيتم تهيئتهم عند الحاجة
        self.embedder = None
        self.index = None
        self.docs = None
        self.client = None

    def initialize(self):
        if self.initialized:
            return

        print("[RAG-GPT] جاري تهيئة النظام...")
        start_time = time.time()

        # 1. تحميل وتجهيز المستندات
        loader = TextLoader(self.document_path, encoding="utf-8")
        documents = loader.load()

        # تنظيف النص
        def clean_text(text):
            text = re.sub(r"[🌟🎯📌💡🩺📞💳❓📜🔒⏰📝*]+", " ", text)
            text = re.sub(r"#+", " ", text)
            text = re.sub(r"\*+", " ", text)
            text = re.sub(r"\s+", " ", text).strip()
            return text

        all_text = " ".join([clean_text(doc.page_content) for doc in documents])

        # تقسيم النص
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=500,
            chunk_overlap=50,
            separators=["\n", " ", "،", "."]
        )
        self.docs = text_splitter.create_documents([all_text])

        # 2. تهيئة الـEmbedder
        self.embedder = SentenceTransformer("OmarAlsaabi/e5-base-mlqa-finetuned-arabic-for-rag")

        # 3. إنشاء FAISS index
        embeddings = np.array([self.embedder.encode(doc.page_content) for doc in self.docs]).astype("float32")
        dimension = embeddings.shape[1]
        self.index = faiss.IndexFlatL2(dimension)
        self.index.add(embeddings)

        # 4. تهيئة OpenAI client
        self.client = OpenAI(
            base_url="https://openrouter.ai/api/v1",
            api_key=self.api_key
        )

        init_time = time.time() - start_time
        print(f"✅ النظام اكتمل في {init_time:.2f} ثانية")
        self.initialized = True

    def retrieve_context(self, query, k=3):
        """استرجاع المعلومات ذات الصلة من المستندات"""
        if not self.initialized:
            self.initialize()

        query_emb = np.array(self.embedder.encode([query])).astype("float32")
        D, I = self.index.search(query_emb, k)
        return " ".join([self.docs[i].page_content for i in I[0]])

    def generate_answer(self, query, context=""):
        """توليد الإجابة باستخدام GPT"""
        if not self.initialized:
            self.initialize()

        prompt = f"""
استنادًا إلى المعلومات التالية:
{context}

السؤال: {query}

الإجابة بالعربية بشكل واضح ومباشر:
"""
        response = self.client.chat.completions.create(
            model="openai/gpt-oss-120b",
            messages=[
                {"role": "system", "content": "أنت مساعد ذكي يجيب بالعربية بناءً على المعلومات المقدمة"},
                {"role": "user", "content": prompt}
            ],
            temperature=0.5,
            max_tokens=400
        )
        return response.choices[0].message.content.strip()

    def ask_question(self, query):
        """طريقة موحدة للسؤال - تجمع بين RAG و GPT"""
        if not self.initialized:
            self.initialize()

        print(f"[RAG-GPT] معالجة السؤال: {query}")
        start_time = time.time()

        # 1. استرجاع المعلومات
        context = self.retrieve_context(query)

        # 2. توليد الإجابة
        answer = self.generate_answer(query, context)

        process_time = time.time() - start_time
        print(f"✅ تمت المعالجة في {process_time:.2f} ثانية")

        return answer

# اختبار الكلاس
if __name__ == "__main__":
    # الإعدادات
    DOCUMENT_PATH = "/content/info.md"  # تأكد من وجود الملف
    API_KEY = "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"  # استبدل بالمفتاح الحقيقي
# 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
    # إنشاء النظام
    rag_gpt = ArabicRAGGPTold(DOCUMENT_PATH, API_KEY)

    # اختبار السؤال
    question = "ما هي المؤهلات التي يتمتع بها الأطباء في شركة الشفاء الرقمية؟"
    answer = rag_gpt.ask_question(question)
    print(f"\n🎯 السؤال: {question}")
    print(f"💡 الإجابة: {answer}")

[RAG-GPT] جاري تهيئة النظام...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/270 [00:00<?, ?B/s]

✅ النظام اكتمل في 38.09 ثانية
[RAG-GPT] معالجة السؤال: ما هي المؤهلات التي يتمتع بها الأطباء في شركة الشفاء الرقمية؟
✅ تمت المعالجة في 3.38 ثانية

🎯 السؤال: ما هي المؤهلات التي يتمتع بها الأطباء في شركة الشفاء الرقمية؟
💡 الإجابة: الأطباء في شركة الشفاء الرقمية جميعهم حاصلون على شهادات طبية معتمدة، ويتمتعون بسنوات من الخبرة العملية في مجال الرعاية الصحية.


In [ ]:
   # اختبار السؤال
question2 = "السلام عليكم حياك الله كيفك"
answer2 = rag_gpt.ask_question(question2)
print(f"\n🎯 السؤال: {question2}")
print(f"💡 الإجابة: {answer2}")

[RAG-GPT] معالجة السؤال: السلام عليكم حياك الله كيفك
✅ تمت المعالجة في 5.44 ثانية

🎯 السؤال: السلام عليكم حياك الله كيفك
💡 الإجابة: وعليكم السلام ورحمة الله وبركاته، أهلاً وسهلاً! أنا بخير، شكرًا لسؤالك. كيف يمكنني مساعدتك اليوم؟


In [ ]:
!pip install torch
!pip install sentence-transformers
!pip install faiss-cpu
!pip install langchain
!pip install openai
!pip install numpy
!pip install regex

In [ ]:
%pip install faiss-cpu

In [ ]:
! pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 45.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.0 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [ ]:
import os, re, time, pickle
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from openai import OpenAI

# ==============================
# 1) كلاس معالجة المستندات
# ==============================
class DocumentProcessor:
    def __init__(self, document_path: str):
        self.document_path = document_path

    def clean_text(self, text: str) -> str:
        """تنظيف النص من الرموز والإيموجي"""
        text = re.sub(r"[🌟🎯📌💡🩺📞💳❓📜🔒⏰📝*]+", " ", text)
        text = re.sub(r"[\"“”]+", "\"", text)
        text = re.sub(r"#+", " ", text)
        text = re.sub(r"\*+", " ", text)
        text = re.sub(r"\s+", " ", text).strip()
        return text

    def load_and_split(self, chunk_size=500, chunk_overlap=50):
        """تحميل المستند وتنظيفه ثم تقسيمه"""
        loader = TextLoader(self.document_path, encoding="utf-8")
        documents = loader.load()

        all_text = " ".join([self.clean_text(doc.page_content) for doc in documents])

        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=chunk_size,
            chunk_overlap=chunk_overlap,
            separators=["\n", " ", "،", "."]
        )
        return text_splitter.create_documents([all_text])

# ==============================
# 2) كلاس قاعدة البيانات (FAISS + Embeddings)
# ==============================
class VectorStore:
    def __init__(self, model_name="OmarAlsaabi/e5-base-mlqa-finetuned-arabic-for-rag"):
        self.embedder = SentenceTransformer(model_name)
        self.index = None
        self.docs = None

    def build_index(self, docs):
        """بناء الفهرس من المستندات"""
        self.docs = docs
        embeddings = np.array([self.embedder.encode(doc.page_content) for doc in docs]).astype("float32")
        dim = embeddings.shape[1]
        self.index = faiss.IndexFlatL2(dim)
        self.index.add(embeddings)

    def save_index(self, index_path="index.faiss", docs_path="docs.pkl"):
        """حفظ الفهرس + النصوص"""
        faiss.write_index(self.index, index_path)
        with open(docs_path, "wb") as f:
            pickle.dump(self.docs, f)

    def load_index(self, index_path="index.faiss", docs_path="docs.pkl"):
        """تحميل الفهرس + النصوص"""
        if os.path.exists(index_path) and os.path.exists(docs_path):
            self.index = faiss.read_index(index_path)
            with open(docs_path, "rb") as f:
                self.docs = pickle.load(f)
            return True
        return False

    def retrieve(self, query, k=3):  # غيرت k=1 إلى k=3
        """استرجاع النصوص الأكثر صلة"""
        query_emb = np.array(self.embedder.encode([query])).astype("float32")
        D, I = self.index.search(query_emb, k)
        return " ".join([self.docs[i].page_content for i in I[0]])

# ==============================
# 3) كلاس RAG مع GPT - معدل
# ==============================
class ArabicRAGGPT:
    def __init__(self, document_path, api_key,
                 index_path="index.faiss", docs_path="docs.pkl"):
        self.document_path = document_path
        self.api_key = api_key
        self.client = OpenAI(base_url="https://openrouter.ai/api/v1", api_key=api_key)

        self.processor = DocumentProcessor(document_path)
        self.vstore = VectorStore()
        self.initialized = False

        self.index_path = index_path
        self.docs_path = docs_path

    def initialize(self):
        if self.initialized:
            return
        print("[RAG-GPT] جاري التهيئة...")
        start = time.time()

        loaded = self.vstore.load_index(self.index_path, self.docs_path)
        if not loaded:
            docs = self.processor.load_and_split()
            self.vstore.build_index(docs)
            self.vstore.save_index(self.index_path, self.docs_path)
            print("[RAG-GPT] تم إنشاء الفهرس وحفظه لأول مرة.")
        else:
            print("[RAG-GPT] تم تحميل الفهرس من الملفات المخزنة.")

        self.initialized = True
        print(f"✅ التهيئة اكتملت في {time.time() - start:.2f} ثانية")

    def generate_answer(self, query, context):
        """توليد إجابة من GPT - معدل زي الكود القديم"""
        prompt = f"""
استنادًا إلى المعلومات التالية:
{context}

السؤال: {query}

الإجابة بالعربية بشكل واضح ومباشر:
"""
        response = self.client.chat.completions.create(
            model="openai/gpt-oss-120b",  # جرب هذا أولاً
            # model="meta-llama/llama-3.1-8b-instruct:free",  # أو جرب هذا مجاناً
            messages=[
                {"role": "system", "content": "أنت مساعد ذكي يجيب بالعربية بناءً على المعلومات المقدمة"},
                {"role": "user", "content": prompt}
            ],
            temperature=0.5,  # رجعت temperature للقديم
            max_tokens=400    # زودت الـ tokens
        )
        return response.choices[0].message.content.strip()

    def ask_question(self, query):
        if not self.initialized:
            self.initialize()

        print(f"[RAG-GPT] سؤال: {query}")
        start = time.time()

        context = self.vstore.retrieve(query, k=3)  # غيرت k=5 إلى k=3
        answer = self.generate_answer(query, context)

        print(f"✅ تمت المعالجة في {time.time() - start:.2f} ثانية")
        return answer

# ==============================
# 4) اختبار
# ==============================
if __name__ == "__main__":
    DOCUMENT_PATH = "/content/info.md"
    API_KEY = "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"

    rag_gpt = ArabicRAGGPT(DOCUMENT_PATH, API_KEY)

    # اختبر بسؤال بسيط أولاً
    question = "ما هي المؤهلات التي يتمتع بها الأطباء في شركة الشفاء الرقمية؟"
    answer = rag_gpt.ask_question(question)
    print("\n🎯 السؤال:", question)
    print("💡 الإجابة:", answer)

[RAG-GPT] جاري التهيئة...
[RAG-GPT] تم تحميل الفهرس من الملفات المخزنة.
✅ التهيئة اكتملت في 0.00 ثانية
[RAG-GPT] سؤال: ما هي المؤهلات التي يتمتع بها الأطباء في شركة الشفاء الرقمية؟
✅ تمت المعالجة في 9.49 ثانية

🎯 السؤال: ما هي المؤهلات التي يتمتع بها الأطباء في شركة الشفاء الرقمية؟
💡 الإجابة: الأطباء في شركة الشفاء الرقمية جميعهم يحملون **شهادات طبية معتمدة** (مثل البكالوريوس أو ما يعادله في الطب) بالإضافة إلى **تخصصات معتمدة** في مجالاتهم (باطنية، أطفال، جلدية، نسائية، عيون، أنف‑أذن‑حنجرة، إلخ). كما يتمتعون **بخبرة عملية لسنوات** في مجال الرعاية الصحية عن بُعد، مما يضمن تقديم استشارات طبية دقيقة ومُعتمدة.


In [ ]:
import os, re, time, pickle
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from openai import OpenAI

# ==============================
# 1) كلاس معالجة المستندات
# ==============================
class DocumentProcessor:
    def __init__(self, document_path: str):
        self.document_path = document_path

    def clean_text(self, text: str) -> str:
        """تنظيف النص من الرموز والإيموجي"""
        # إزالة كل الرموز والإيموجي والعلامات
        text = re.sub(r"[🌟🎯📌💡🩺📞💳❓📜🔒⏰📝*#\-]+", " ", text)
        text = re.sub(r"[\"“”]+", "\"", text)
        text = re.sub(r"\s+", " ", text).strip()
        return text

    def load_and_split(self, chunk_size=500, chunk_overlap=50):
        """تحميل المستند وتنظيفه ثم تقسيمه"""
        loader = TextLoader(self.document_path, encoding="utf-8")
        documents = loader.load()

        # تنظيف النص تماماً قبل التقسيم
        cleaned_texts = []
        for doc in documents:
            cleaned_text = self.clean_text(doc.page_content)
            # إزالة الأسطر الفارغة والنصوص القصيرة جداً
            if cleaned_text and len(cleaned_text) > 10:
                cleaned_texts.append(cleaned_text)

        all_text = " ".join(cleaned_texts)

        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=chunk_size,
            chunk_overlap=chunk_overlap,
            separators=["\n\n", "\n", "،", ".", " "]
        )
        chunks = text_splitter.create_documents([all_text])

        # تنظيف إضافي للـ chunks
        cleaned_chunks = []
        for chunk in chunks:
            clean_content = self.clean_text(chunk.page_content)
            if clean_content and len(clean_content) > 20:  # تجاهل النصوص القصيرة
                cleaned_chunks.append(clean_content)

        return text_splitter.create_documents(cleaned_chunks)

# ==============================
# 2) كلاس قاعدة البيانات (FAISS + Embeddings)
# ==============================
class VectorStore:
    def __init__(self, model_name="OmarAlsaabi/e5-base-mlqa-finetuned-arabic-for-rag"):
        self.embedder = SentenceTransformer(model_name)
        self.index = None
        self.docs = None
        self.embeddings = None  # حفظ الـ embeddings

    def build_index(self, docs):
        """بناء الفهرس من المستندات"""
        self.docs = docs
        print(f"[VectorStore] جاري إنشاء embeddings لـ {len(docs)} جزء...")

        # إنشاء embeddings مرة واحدة وحفظها
        self.embeddings = np.array([self.embedder.encode(doc.page_content) for doc in docs]).astype("float32")
        dim = self.embeddings.shape[1]
        self.index = faiss.IndexFlatL2(dim)
        self.index.add(self.embeddings)
        print(f"[VectorStore] تم إنشاء الفهرس بـ {len(docs)} جزء")

    def save_index(self, index_path="index.faiss", docs_path="docs.pkl", emb_path="embeddings.npy"):
        """حفظ الفهرس + النصوص + الـ embeddings"""
        faiss.write_index(self.index, index_path)
        with open(docs_path, "wb") as f:
            pickle.dump(self.docs, f)
        np.save(emb_path, self.embeddings)
        print(f"[VectorStore] تم حفظ الفهرس والبيانات")

    def load_index(self, index_path="index.faiss", docs_path="docs.pkl", emb_path="embeddings.npy"):
        """تحميل الفهرس + النصوص + الـ embeddings"""
        if all(os.path.exists(path) for path in [index_path, docs_path, emb_path]):
            self.index = faiss.read_index(index_path)
            with open(docs_path, "rb") as f:
                self.docs = pickle.load(f)
            self.embeddings = np.load(emb_path)
            print(f"[VectorStore] تم تحميل الفهرس بـ {len(self.docs)} جزء")
            return True
        return False

    def retrieve(self, query, k=3):
        """استرجاع النصوص الأكثر صلة"""
        query_emb = np.array(self.embedder.encode([query])).astype("float32")
        D, I = self.index.search(query_emb, k)

        # جمع النصوص ذات الصلة
        relevant_texts = []
        for i in I[0]:
            if i < len(self.docs):  # تجنب الأخطاء
                clean_text = self.clean_context(self.docs[i].page_content)
                relevant_texts.append(clean_text)

        return " ".join(relevant_texts)

    def clean_context(self, text):
        """تنظيف النص قبل العرض"""
        text = re.sub(r"\*+", " ", text)  # إزالة العلامات
        text = re.sub(r"\s+", " ", text).strip()
        return text

# ==============================
# 3) كلاس RAG مع GPT - النسخة النهائية
# ==============================
class ArabicRAGGPT_Newone:
    def __init__(self, document_path, api_key,
                 index_path="index.faiss", docs_path="docs.pkl", emb_path="embeddings.npy"):
        self.document_path = document_path
        self.api_key = api_key
        self.client = OpenAI(base_url="https://openrouter.ai/api/v1", api_key=api_key)

        self.processor = DocumentProcessor(document_path)
        self.vstore = VectorStore()
        self.initialized = False

        self.index_path = index_path
        self.docs_path = docs_path
        self.emb_path = emb_path

    def initialize(self):
        if self.initialized:
            return
        print("[RAG-GPT] جاري التهيئة...")
        start = time.time()

        loaded = self.vstore.load_index(self.index_path, self.docs_path, self.emb_path)
        if not loaded:
            print("[RAG-GPT] جاري معالجة المستند لأول مرة...")
            docs = self.processor.load_and_split()
            print(f"[RAG-GPT] تم تقسيم المستند إلى {len(docs)} جزء")
            self.vstore.build_index(docs)
            self.vstore.save_index(self.index_path, self.docs_path, self.emb_path)
            print("[RAG-GPT] تم إنشاء الفهرس وحفظه لأول مرة.")
        else:
            print("[RAG-GPT] تم تحميل الفهرس من الملفات المخزنة.")

        self.initialized = True
        print(f"✅ التهيئة اكتملت في {time.time() - start:.2f} ثانية")

    def generate_answer(self, query, context):
        """توليد إجابة من GPT - نظيفة وواضحة"""
        prompt = f"""
المعلومات المرجعية:
{context}

السؤال: {query}

المطلوب: قدم إجابة واضحة ومباشرة باللغة العربية بناءً على المعلومات المذكورة أعلاه فقط.
"""
        response = self.client.chat.completions.create(
            model="openai/gpt-oss-120b",
            messages=[
                {"role": "system", "content": "أنت مساعد عربي محترف. قدم إجابات دقيقة وواضحة بناءً على المعلومات المعطاة."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.3,  # أقل لنتائج أكثر استقراراً
            max_tokens=500
        )
        return self.clean_answer(response.choices[0].message.content.strip())

    def clean_answer(self, answer):
        """تنظيف الإجابة النهائية"""
        # إزالة العلامات والرموز غير المرغوبة
        answer = re.sub(r"\*+", "", answer)
        answer = re.sub(r"#+", "", answer)
        answer = re.sub(r"-+", " - ", answer)
        answer = re.sub(r"\s+", " ", answer).strip()
        return answer

    def ask_question(self, query):
        """طرح سؤال والحصول على إجابة نظيفة"""
        if not self.initialized:
            self.initialize()

        print(f"\n[RAG-GPT] السؤال: {query}")
        start = time.time()

        context = self.vstore.retrieve(query, k=3)
        answer = self.generate_answer(query, context)

        process_time = time.time() - start
        print(f"✅ تمت المعالجة في {process_time:.2f} ثانية")

        return answer

# ==============================
# 4) اختبار نهائي
# ==============================
if __name__ == "__main__":
    DOCUMENT_PATH = "/content/info.md"
    API_KEY = "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"

    # إنشاء النظام
    rag_gpt = ArabicRAGGPT_Newone(DOCUMENT_PATH, API_KEY)

    # أسئلة اختبارية
    questions = [
        "ما هي المؤهلات التي يتمتع بها الأطباء في شركة الشفاء الرقمية؟",
        "ما هي خدمات الشركة؟",
        "ما هي أوقات عمل الشركة؟"
    ]

    for i, question in enumerate(questions, 1):
        print(f"\n{'='*50}")
        print(f"السؤال {i}: {question}")
        print(f"{'='*50}")

        answer = rag_gpt.ask_question(question)
        print(f"\nالإجابة: {answer}")

        if i < len(questions):
            print("\n" + "-"*30 + " انتظار السؤال التالي " + "-"*30)


السؤال 1: ما هي المؤهلات التي يتمتع بها الأطباء في شركة الشفاء الرقمية؟
[RAG-GPT] جاري التهيئة...
[RAG-GPT] جاري معالجة المستند لأول مرة...
[RAG-GPT] تم تقسيم المستند إلى 38 جزء
[VectorStore] جاري إنشاء embeddings لـ 38 جزء...
[VectorStore] تم إنشاء الفهرس بـ 38 جزء
[VectorStore] تم حفظ الفهرس والبيانات
[RAG-GPT] تم إنشاء الفهرس وحفظه لأول مرة.
✅ التهيئة اكتملت في 1.08 ثانية

[RAG-GPT] السؤال: ما هي المؤهلات التي يتمتع بها الأطباء في شركة الشفاء الرقمية؟
✅ تمت المعالجة في 7.57 ثانية

الإجابة: الأطباء في شركة الشفاء الرقمية يتمتعون بالمؤهلات التالية: - شهادات معتمدة في تخصصاتهم الطبية. - سنوات من الخبرة العملية في المجال الذي يختصون به. - خضوعهم لعملية تدقيق صارمة لضمان مستويات الكفاءة والاحترافية. هذه المتطلبات تضمن أن جميع الأطباء في الشركة مؤهلون لتقديم رعاية طبية عالية الجودة.

------------------------------ انتظار السؤال التالي ------------------------------

السؤال 2: ما هي خدمات الشركة؟

[RAG-GPT] السؤال: ما هي خدمات الشركة؟
✅ تمت المعالجة في 7.90 ثانية

الإجابة: خدمات الشركة وف

In [ ]:
if __name__ == "__main__":
    DOCUMENT_PATH = "/content/info.md"
    API_KEY = "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"

    # إنشاء النظام
    rag_gpt = ArabicRAGGPT_Newone(DOCUMENT_PATH, API_KEY)

    # أسئلة اختبارية
    questions = [
        "ما هي المؤهلات التي يتمتع بها الأطباء في شركة الشفاء الرقمية؟",
        "ما هي خدمات الشركة؟",
        "ما هي أوقات عمل الشركة؟",
        "لو هلغي الاشتراك المفروض اعمل اي ؟",
         "ممكن اسالك سوال تاني لو عاوز اللغي الاشتراك اعمل اي او اي الاجراءات اللزمه؟"
    ]

    for i, question in enumerate(questions, 1):
        print(f"\n{'='*50}")
        print(f"السؤال {i}: {question}")
        print(f"{'='*50}")

        answer = rag_gpt.ask_question(question)
        print(f"\nالإجابة: {answer}")

        if i < len(questions):
            print("\n" + "-"*30 + " انتظار السؤال التالي " + "-"*30)


السؤال 1: ما هي المؤهلات التي يتمتع بها الأطباء في شركة الشفاء الرقمية؟
[RAG-GPT] جاري التهيئة...
[VectorStore] تم تحميل الفهرس بـ 38 جزء
[RAG-GPT] تم تحميل الفهرس من الملفات المخزنة.
✅ التهيئة اكتملت في 0.00 ثانية

[RAG-GPT] السؤال: ما هي المؤهلات التي يتمتع بها الأطباء في شركة الشفاء الرقمية؟
✅ تمت المعالجة في 1.21 ثانية

الإجابة: الأطباء في شركة الشفاء الرقمية يتمتعون بالمؤهلات التالية: - شهادات معتمدة في تخصصاتهم الطبية. - خبرة عملية تمتد لسنوات عديدة في مجال التخصص. - خضوعهم لعملية تدقيق صارمة لضمان أعلى مستويات الكفاءة والاحترافية. هذه المؤهلات تضمن تقديم رعاية طبية عالية الجودة ومهنية للمرضى.

------------------------------ انتظار السؤال التالي ------------------------------

السؤال 2: ما هي خدمات الشركة؟

[RAG-GPT] السؤال: ما هي خدمات الشركة؟
✅ تمت المعالجة في 2.85 ثانية

الإجابة: خدمات الشركة وفقًا للمعلومات المذكورة هي: 1. الباقات والعضويات - خيارات اشتراك شهرية أو سنوية. - توفر خصومات إضافية ووصولًا غير محدود إلى بعض الخدمات. 2. الدعم الفني - الدردشة المباشرة (Live Chat): م

In [ ]:
if __name__ == "__main__":
    DOCUMENT_PATH = "/content/info.md"
    API_KEY = "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"

    # إنشاء النظام
    rag_gpt = ArabicRAGGPT_Newone(DOCUMENT_PATH, API_KEY)

    # أسئلة اختبارية
    questions = [
      "مسموح استرد فلوس الاشتراك بعد مده قد اي ؟"
    ]

    for i, question in enumerate(questions, 1):
        print(f"\n{'='*50}")
        print(f"السؤال {i}: {question}")
        print(f"{'='*50}")

        answer = rag_gpt.ask_question(question)
        print(f"\nالإجابة: {answer}")

        if i < len(questions):
            print("\n" + "-"*30 + " انتظار السؤال التالي " + "-"*30)


السؤال 1: مسموح استرد فلوس الاشتراك بعد مده قد اي ؟
[RAG-GPT] جاري التهيئة...
[VectorStore] تم تحميل الفهرس بـ 38 جزء
[RAG-GPT] تم تحميل الفهرس من الملفات المخزنة.
✅ التهيئة اكتملت في 0.00 ثانية

[RAG-GPT] السؤال: مسموح استرد فلوس الاشتراك بعد مده قد اي ؟
✅ تمت المعالجة في 1.52 ثانية

الإجابة: نعم، يمكنك طلب استرداد مبلغ الاشتراك فقط خلال 24 ساعة من تقديم الخدمة أو في حال عدم إتمامها. بعد مرور هذه المدة لا يُسمح بطلب الاسترداد وفقاً لسياسة الاسترداد المذكورة.
